#nome:Gabriel Silva rm:552397 turma:2 tiar
#nome: Lucas Caspirro Gitti Alcaraz rm:551090 turma :2tiar

#imports

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import zipfile
import os
import pandas as pd
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


#preparando o arquivo zip

In [ ]:
# Caminho do arquivo ZIP
zip_path = '/content/archive (1).zip'
extract_path = '/content/archive'

# Extrair o conteúdo do arquivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

#preparando as imagens

In [ ]:
# Caminho para as pastas extraídas
healthy_path = Path(extract_path) / 'health_corals'
diseased_path = Path(extract_path) / 'bleached_corals'

# Criar listas de caminhos e labels
data = []
for img_path in healthy_path.glob('*.jpg'):
    data.append((str(img_path), 'saudavel'))
for img_path in diseased_path.glob('*.jpg'):
    data.append((str(img_path), 'doente'))

# Criar DataFrame
df = pd.DataFrame(data, columns=['image_path', 'label'])

# Verificar se o DataFrame foi criado corretamente
print(df['label'].value_counts())


label
doente    483
Name: count, dtype: int64


#separação de treino e teste

In [ ]:
# Definir gerador de dados
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Caminho para o diretório principal
main_dir = extract_path

# Criar geradores de dados para treinamento e validação
train_generator = datagen.flow_from_directory(
    main_dir,
    target_size=(150, 150),  # tamanho das imagens
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    main_dir,
    target_size=(150, 150),  # tamanho das imagens
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)


Found 739 images belonging to 2 classes.
Found 183 images belonging to 2 classes.


#preparação da rede

In [ ]:
# Definir o modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    GlobalAveragePooling2D(),

    Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(1, activation='sigmoid')
])
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#primeiro treinamento

In [ ]:
# Treinar o modelo
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
24/24 [==============================] - 71s 3s/step - loss: 0.8943 - accuracy: 0.6319 - val_loss: 0.8199 - val_accuracy: 0.5246
Epoch 2/10
24/24 [==============================] - 60s 3s/step - loss: 0.8761 - accuracy: 0.6414 - val_loss: 2.4893 - val_accuracy: 0.5246
Epoch 3/10
24/24 [==============================] - 60s 3s/step - loss: 0.8596 - accuracy: 0.6279 - val_loss: 1.8780 - val_accuracy: 0.5246
Epoch 4/10
24/24 [==============================] - 63s 3s/step - loss: 0.8348 - accuracy: 0.6401 - val_loss: 1.3718 - val_accuracy: 0.5246
Epoch 5/10
24/24 [==============================] - 60s 3s/step - loss: 0.7491 - accuracy: 0.6631 - val_loss: 0.6506 - val_accuracy: 0.6230
Epoch 6/10
24/24 [==============================] - 61s 3s/step - loss: 0.7812 - accuracy: 0.6225 - val_loss: 1.1739 - val_accuracy: 0.5246
Epoch 7/10
24/24 [==============================] - 63s 3s/step - loss: 0.7841 - accuracy: 0.6306 - val_loss: 0.7513 - val_accuracy: 0.5519
Epoch 8/10
24/24 [==

#segundo treinamento para salvar o melhor modelo feito pela rede

In [ ]:
# Callback para salvar o melhor modelo
checkpoint = ModelCheckpoint('melhor_modelo.h5', monitor='val_loss', save_best_only=True, mode='min')

# Callback para interromper o treinamento quando não houver melhoria
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

# Treinar o modelo com os callbacks
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/50
24/24 [==============================] - 6s 75ms/step - loss: 0.5908 - accuracy: 0.8904 - val_loss: 1.0259 - val_accuracy: 0.6284
Epoch 2/50
24/24 [==============================] - 2s 71ms/step - loss: 0.5872 - accuracy: 0.9012 - val_loss: 1.0686 - val_accuracy: 0.7104
Epoch 3/50
24/24 [==============================] - 2s 66ms/step - loss: 0.5786 - accuracy: 0.8999 - val_loss: 1.1937 - val_accuracy: 0.6667
Epoch 4/50
24/24 [==============================] - 2s 66ms/step - loss: 0.4954 - accuracy: 0.9188 - val_loss: 1.1580 - val_accuracy: 0.5574
Epoch 5/50
24/24 [==============================] - 2s 63ms/step - loss: 0.5108 - accuracy: 0.9093 - val_loss: 1.1451 - val_accuracy: 0.6120
Epoch 6/50
24/24 [==============================] - 2s 66ms/step - loss: 0.6864 - accuracy: 0.8904 - val_loss: 2.3616 - val_accuracy: 0.4918
Epoch 7/50
24/24 [==============================] - 2s 99ms/step - loss: 0.6625 - accuracy: 0.9053 - val_loss: 1.2602 - val_accuracy: 0.7104
Epoch 8/50
24

#acuracia

In [ ]:
# Avaliar o modelo nos dados de validação
loss, accuracy = model.evaluate(validation_generator)
print(f'Acurácia no conjunto de validação: {accuracy * 100:.2f}%')


6/6 [==============================] - 0s 62ms/step - loss: 0.9836 - accuracy: 0.7650
Acurácia no conjunto de validação: 76.50%


#salvando o modelo

In [ ]:
model = load_model('melhor_modelo.h5')